# Test Model

This is the test notebook for the JPL-Code-Sample and it will generate both `results.csv` and `output`.

## Generated Content

`results.csv` contains the the percentages for each class type.

`output` is the directory that contains all the image visualizations.

## Results

The `results.csv` is formatted as such: Each row is an image which has the corresponding filename.  The percentages for each class is in contained in the same column with its respective class name.

## Image Visualizations

There are three generated images in the output folder for each original testing image.

1. Original Image (no prefix) (left)
2. Class Label Colors (mask prefix) (middle)
3. Transparent label mask (overlay prefix) (right)


<img src="samples/original.jpeg" style="width: 200px;" align="left">

<img src="samples/mask.jpeg" style="width: 200px;" align="left">

<img src="samples/overlay.jpeg" style="width: 200px;" align="left">


### Imports

In [13]:
from __future__ import division
from __future__ import print_function

import utils
import os
import cv2
import numpy as np
import pandas as pd

from keras.models import model_from_json
from scipy import ndimage 

### Set Parameters

In [14]:
# Size of training image files.
image_size = (512,512)

# The test image path
TEST_PATH = "data/test/"

# The output path for visualizations.
OUTPUT_PATH = "output"

# The number of slices each image will be cut into.
n_slices = 32

# The ratio of the image.
ratio = (1, 1)

# The opacity of the class label color mask.
mask_opacity = 0.3

classes = ['vegetation', 'water', 'desert', 'clouds']

# The path to save the model.
MODEL_PATH = "models"

# The name the model will be saved to.
model_name = "model"

### Load Model

In [15]:
# load json and create model
json_file = open(os.path.join(MODEL_PATH, model_name + ".json"), 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

# load weights into new model
model.load_weights(os.path.join(MODEL_PATH, model_name + ".h5"))
print("Loaded model from disk")

Loaded model from disk


### Load Test data

In [16]:
# Read the URL for test data.
with open("test_images.txt", 'r') as f:
    image_urls = f.read().split("\n")

print("Downloading test images...")    

if not os.path.exists(TEST_PATH):
    os.mkdir(TEST_PATH)

# Download the test images.
for i, image_url in enumerate(image_urls):
    utils.download_image(image_url, os.path.join(TEST_PATH, '_'.join(image_url.split('/')[8:])))
    
print("Preparing test data.")

# Prepare the test data.
X_test = []
full_images = []
test_image_f_names = os.listdir(TEST_PATH)

# Load all files.
for f_name in test_image_f_names:
    img = cv2.imread(os.path.join(TEST_PATH, f_name))    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resize = cv2.resize(img, image_size)
    
    # Preprocess test data.
    slices = utils.prepare_images(img_resize, image_size, ratio, n_slices)
    X_test.append(slices)
    full_images.append(img_resize)
    
# Collect and format all test data.
full_images = np.stack(full_images)
X_test = np.stack(X_test)
X_test = X_test.reshape(X_test.shape[0] * X_test.shape[1], X_test.shape[2], X_test.shape[3], X_test.shape[4])

Preparing test data.


### Label classification

In [17]:
# Predict class labels for slices.
preds = model.predict(X_test, verbose=1)

57280/57344 [============================>.] - ETA: 0s

In [18]:
# The maximum probability of the model is the class label.
labels = np.argmax(preds, axis=1)

# Reshape the labels for results and visualization purposes.
labels = np.reshape(labels, (len(full_images), n_slices, n_slices))

### Generate results

In [22]:
# Initialize list to hold label percentages
percentages = []

for l in labels:
    percentages.append(np.bincount(l.flatten()) / l.size)
    
# Collect percentages into dataframe and save to disk.
results = pd.DataFrame(percentages, columns=classes)
results['f_name'] = test_image_f_names
results = results.fillna(0)
results.to_csv("results.csv")

In [25]:
results

,vegetation,water,desert,clouds,f_name
0,0.000000,0.955078,0.000000,0.044922,2018-03-23_250m_5_11_19.jpeg
1,0.019531,0.048828,0.927734,0.003906,2018-03-23_250m_5_7_23.jpeg
2,0.002930,0.493164,0.335938,0.167969,2018-03-23_250m_5_10_20.jpeg
3,0.003906,0.031250,0.960938,0.003906,2018-03-23_250m_5_7_18.jpeg
4,0.000000,0.000000,1.000000,0.000000,2018-03-23_250m_5_7_21.jpeg
5,0.143555,0.000977,0.846680,0.008789,2018-03-23_250m_5_8_19.jpeg
6,0.832031,0.002930,0.116211,0.048828,2018-03-23_250m_5_9_23.jpeg
7,0.000000,0.861328,0.000000,0.138672,2018-03-23_250m_5_13_18.jpeg
8,0.000000,0.911133,0.000000,0.088867,2018-03-23_250m_5_12_18.jpeg
9,0.430664,0.476562,0.055664,0.037109,2018-03-23_250m_5_10_24.jpeg


### Visualization

In [20]:
# Assign color for each class.

# Vegetation: Green
# Water: Blue
# Desert: Brown
# Clouds: White

colors = {
    0:(124,252,0),
    1:(244,164,96),
    2:(0,191,255),
    3:(255, 255, 255)
}

In [21]:
if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

# Iterate over all test data and create visualizations.
for i, (f_name, img, l) in enumerate(zip(test_image_f_names, full_images, labels)):
    
    # Initialize the class assignment mask.
    assignment_mask = np.zeros((l.shape[0], l.shape[1], 3))
    
    # Fill class labels into assignment mask.
    for label in xrange(len(classes)):
        assignment_mask[np.isin(l, label)] = colors[label]
    
    # Convert image to BGR to see correct color channels in visualization.
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    

    # assignment mask needs to be of type uint8 to lay mask over image.
    output = img.copy()
    assignment_mask = assignment_mask.astype('uint8')
    
    # Flip and rotate assignment mask to account for offset.
    assignment_mask = cv2.resize(assignment_mask, image_size, interpolation=cv2.INTER_CUBIC)
    assignment_mask = cv2.flip(ndimage.rotate(assignment_mask, 90),0)
    
    # Apply mask to image.
    overlay = assignment_mask
    output = cv2.addWeighted(overlay, mask_opacity, output, 1 - mask_opacity,
                    0, overlay)
    
    # Save images to disk.
    cv2.imwrite(os.path.join(OUTPUT_PATH, f_name), img)
    cv2.imwrite(os.path.join(OUTPUT_PATH, "overlay_" + f_name), overlay)
    cv2.imwrite(os.path.join(OUTPUT_PATH, "mask_" + f_name), assignment_mask)